In [1]:
import sys
sys.path.append('../')

from config import corpus_path

import cohesion_tokenizer
from cohesion_tokenizer import CohesionScore
from cohesion_tokenizer import WordSequenceGraph

In [2]:
print('corpus name = {}'.format(corpus_path.split('/')[-1]))

corpus name = 2016-10-20_article_all_normed.txt


In [3]:
with open(corpus_path, encoding='utf-8') as f:
    sents = [sent.strip() for doc in f for sent in doc.split('  ')]
print('num sents = {}'.format(len(sents)))

num sents = 223435


In [4]:
import pickle
# cohesion = CohesionScore()
# cohesion.train(sents)
# with open('./cohesion.pkl', 'wb') as f:
#     pickle.dump(cohesion, f)
with open('./cohesion.pkl', 'rb') as f:
    cohesion = pickle.load(f)

In [5]:
for word in '아이오아이 뉴스 대통령 아이돌'.split():
    print('{} = {}'.format(word, cohesion[word]))

아이오아이 = 0.30039353721795514
뉴스 = 0.4855283438945025
대통령 = 0.2919357101768181
아이돌 = 0.09792934452242717


In [6]:
word = '아이오아이'
n = len(word)
for i in range(1, n+1):
    print('{} = {}'.format(word[:i], cohesion[word[:i]]))

아 = 0
아이 = 0.14807442926505623
아이오 = 0.0962206918738307
아이오아 = 0.20118122417377401
아이오아이 = 0.30039353721795514


In [7]:
cost_func = lambda word, word_score: len(word) * (1 - word_score)
graph = WordSequenceGraph(cohesion)

sent = '아이오아이가뮤직뱅크에출연했습니다'
edges = graph.as_graph(sent)

In [8]:
from shortestpath import list_to_dict_graph

g = list_to_dict_graph(edges)

In [9]:
g[('BOS', )]

{('아', 0, 1): 1,
 ('아이', 0, 2): 1.7038511414698876,
 ('아이오', 0, 3): 2.7113379243785083,
 ('아이오아', 0, 4): 3.195275103304904,
 ('아이오아이', 0, 5): 3.4980323139102243,
 ('아이오아이가', 0, 6): 4.6662161132798365}

In [10]:
for word in '뮤직뱅크 뮤직 뱅크'.split():
    print('{} = {}'.format(word, cohesion[word]))

뮤직뱅크 = 0.2211725642420726
뮤직 = 0.58887171561051
뱅크 = 0.8176470588235294


In [11]:
from shortestpath import ford

bos = ('BOS',)
eos = ('EOS',)
ford(g, bos, eos)

{'cost': 13.582573597695621,
 'paths': [[('BOS',),
   ('아이오아이', 0, 5),
   ('가', 5, 6),
   ('뮤직', 6, 8),
   ('뱅크', 8, 10),
   ('에', 10, 11),
   ('출연', 11, 13),
   ('했습니다', 13, 17),
   ('EOS',)]]}

In [12]:
from cohesion_tokenizer import CohesionGraphWordSegmentor

word_segmentor = CohesionGraphWordSegmentor()
word_segmentor.train(sents)

training was done with 223435 sents... 


In [13]:
word_segmentor.is_trained

True

In [14]:
word_segmentor.tokenize(sent)

{'cost': 13.582573597695621,
 'paths': [[('BOS',),
   ('아이오아이', 0, 5),
   ('가', 5, 6),
   ('뮤직', 6, 8),
   ('뱅크', 8, 10),
   ('에', 10, 11),
   ('출연', 11, 13),
   ('했습니다', 13, 17),
   ('EOS',)]]}

In [15]:
import soynlp
print(soynlp.__version__)

from soynlp.tokenizer import MaxScoreTokenizer

tokenizer = MaxScoreTokenizer(scores = cohesion._scores)
tokenizer.tokenize(sent)

0.0.46


['아이오아이', '가', '뮤직', '뱅크', '에', '출연', '했습니다']

In [16]:
tokenizer.tokenize(sent, flatten=False)

[[('아이오아이', 0, 5, 0.30039353721795514, 5),
  ('가', 5, 6, 0.0, 1),
  ('뮤직', 6, 8, 0.58887171561051, 2),
  ('뱅크', 8, 10, 0.8176470588235294, 2),
  ('에', 10, 11, 0.0, 1),
  ('출연', 11, 13, 0.271523178807947, 2),
  ('했습니다', 13, 17, 0.3898437024326576, 4)]]